In [48]:
import urllib.request
import random
import time
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import os

In [49]:
t = random.uniform(0, 7)

In [50]:
def get_page_html(page_url):
    try:
        with urllib.request.urlopen(page_url) as response:
            html = response.read().decode('utf-8')
            soup = BeautifulSoup(html, 'html.parser')
    
    except HTTPError as e:
        print('Error code: ', e.code)
        pass
    except URLError as e:
        print('Reason: ', e.reason)
        pass
    
    return soup

In [51]:
def get_data(soup, page_url):
    title = soup.find('span', attrs={'class':'title-info-title-text'})
    if title:
        adv_title = title.text.strip()
    else:
        adv_title = None
    
    date = soup.find('div', attrs={'class':'title-info-metadata-item'})
    if date:
        adv_date = date.text.strip()
    else:
        adv_date = None
    
    price = soup.find('span', attrs={'class':'price-value-string js-price-value-string'})
    if price:
        adv_price = price.text.strip()
    else:
        adv_price = None
    
    address = soup.find('span', attrs={'class':'item-map-address'})
    if address:
        adv_address = address.text.strip()
    else:
        adv_address = None
    
    main_text = soup.find('div', attrs={'class':'item-description-text'})
    if main_text:
        adv_text = main_text.text.strip()
    else:
        adv_text = None
    
    adv_data = str()
    
    adv_data = 'Title: ' + str(adv_title) + '\n' + 'Date: ' + str(adv_date) + '\n' + \
                'Price: ' + str(adv_price) + '\n' + 'Address: ' + str(adv_address) + '\n' + str(adv_text)
    
    return adv_data

In [52]:
def write_data(adv_data, filename):
    if not os.path.exists('Avito'):
        os.makedirs('Avito')
    filename = filename.replace('/', '\\')
    with open('Avito' + os.sep + filename + '.txt', 'w', encoding='utf-8') as f:
        f.write(adv_data)

In [ ]:
for i in tqdm(range(1, 101)):
    time.sleep(t)
    common_url = 'https://www.avito.ru/moskva/igry_pristavki_i_programmy?p=%s&view=list' % (i)
    page_soup = get_page_html(common_url)
    advs = page_soup.find_all('a', attrs={'class':'description-title-link'})
    advs_links = ['https://www.avito.ru' + link['href'] for link in advs]
    time.sleep(t)
    for adv_link in advs_links:
        adv_soup = get_page_html(adv_link)
        adv_data = get_data(adv_soup, adv_link)
        write_data(adv_data, adv_link)
#         time.sleep(t)

In [57]:
paths = (os.path.join(root, filename)
        for root, _, filenames in os.walk('Avito')
        for filename in filenames)

for path in paths:
    newname = path.replace('\w', 'w')
    if newname != path:
        os.rename(path, newname)